In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>")) #$\color{red}{\text{ciao}}$

# Python para Finanzas y Ciencia de Datos
____

1. Statistics Concepts
2. Portfolio Optimization
3. Principal Component Analysis (PCA)
4. Value-at-Risk (VaR)
____

In [7]:
# IMPORTAMOS LAS LIBRERÍAS QUE VAMOS A USAR A LO LARGO DEL MÓDULO.
import numpy as np
import pandas as pd
import pandas_datareader.data as reader

from matplotlib.ticker import FuncFormatter
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta
from numpy.random import multivariate_normal
import yfinance as yf

import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy.stats import gaussian_kde

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import seaborn as sns

#import functions as f 

import random
random.seed(1000)


# Risk measures
---
Consider three different type of assets, all with the same volatility and mean, but different distributions (e.g.: normal, uniform, Student-t). Standard mean variance analysis indicates that all three assets are equally risky and preferable because their means and volatilities are the same. However, in reality, market participants view the risk in them quite differently and are likely to have a preference for one of these assets. This suggests that some objective way to compare the risk in assets with different distribution is desirable.

Volatility or **standard deviation** is the main measure of risk in most financial analysis, but it **is sufficient as a risk measure only when returns are normally distributed**. However, as discused before, the normality assumption is **violeted for most if not all financial returns**. 


## Value-at-risk (VaR)

The most common risk measure after volatility is **value-at-risk (VaR)**. It is a single summary statistical measure of risk, it is distribution independent and it is a measure of losses as a result of "typical" market movements.

VaR has become one of the most widely used risk measures, and a much debated one. Loved by practitioners for its intuitive appeal, it is also discussed and criticized by many - mainly on theoretical grounds, with regard to its limited ability to capture what is called *tail risk*.

#### Definition: 
***The loss on a trading portfolio such that there is a probability $p$ of losses equaling or exceeding VaR in a given trading period and a $(1-p)$ probability of losses being lower than the VaR (i.e., the confidence level).***

We may write it as $VaR(p)$ or $VaR$ at $100 \times  p \%$ to make the dependence on probability explicit. The most common probability levels are 1% or 5%.

VaR is a quantile of the distribution of profit and loss (P&L). We indicate P&L on an investment portfolio by the random variable $Q$, with a particular realization indicated by $q$. The density of P/L is denoted by $f_q (.)$. Thus, VaR is given by:

$$p = Pr[Q \leq -VaR(p)]$$

or

$$p = \int_{-\infty}^{-VaR(p)}f_q(x)dx$$

We use a minus sign because VaR is a positive number and we are dealing with losses. You should then interpret it as *the probability of losses being larger than VaR*.


#### Example:

Consider a stock position, worth 1 million USD today, that has a $VaR(0.05)$ (or $VaR$ at $5\%$) of 10000 USD over a time period of 30 days (one month). This VaR says that the loss to be expected over a period of 30 days can exceed 50,000 USD with probability 0.01. In other words, there is a probability of 0.99 that losses will not exceed that value. 
However, it does not say anything about the size of the loss once a loss beyond 50,000 USD occurs—i.e., if the maximum loss is 100,000 or 500,000 USD what the probability of such a specific “higher than VaR loss” is. That is what is called the tail risk.

<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/VaR_diagram.JPG' width="60%" align="center"/>


We can study VaR via two methods:

1. Historical distribution
2. Monte Carlo simulations


## 1. Historical distribution 

The issue here is: what is the probability distibution of P&L of a portfolio? The above discussion was based on the assumption that that distribution was known. Howerever, in practice, one needs to estimate the P&L distribution using historical observations of the asset returns of interest. This historical simulation is a simple method for forecasting risk and relies on the assumption that history repeats itself, where one of the observed past returns is expected to be the next period return.

Let's take some of the fund returns we calculated above and calcule the VaR. 
Suppose we invest 10,000 for 22 days. We are interested in the VaR of a financial position in the S&P 500 and the iShares US Treas 10y-20y fund for 22 days.

We use **scs.scoreatpercentile** to get the VaR. **scoreatpercentile(a)** calculates the score at a given percentile of the input sequence **a**.

In [ ]:
initial_investment = 10000
investment_horizon = 22
SP500_prices = data['S&P 500']
US_treas_prices= data['iShares US Treas 10y-20y']

In [ ]:
SP500_prices1 = SP500_prices[:-investment_horizon]
US_treas_prices1 = US_treas_prices[:-investment_horizon]

SP500_prices2 = SP500_prices[investment_horizon:]
US_treas_prices2 = US_treas_prices[investment_horizon:]

In [ ]:
SP500 = (SP500_prices2.values/SP500_prices1.values - 1)*initial_investment
US_treas = (US_treas_prices2.values/US_treas_prices1.values -1)*initial_investment

In [ ]:
SP500 = pd.Series(SP500, name = 'S&P 500')
US_treas = pd.Series(US_treas, name = 'US Treas 10y-20y')

In [ ]:
def plot_var(portfolio, p):
    '''
    Esta función grafica la distribución de P&L y encuentra el VaR para
    un nivel p%.
    Además, 
    '''
    VaR = scs.scoreatpercentile(portfolio, p)
    
    fig = plt.figure(figsize = (18,7))
     
    plt.hist(portfolio, bins = 60)
    plt.xlabel('P&L')
    plt.ylabel('Frequency')
    
    plt.axvline(VaR, color='r', linestyle='dashed', linewidth=2, 
                label = 'VaR at '+str(p)+'% =' +str(round(VaR,2)))
    plt.text(VaR- np.std(portfolio), 100, 
             'p = '+str(p/100), fontsize = 12)
    plt.title('Distribution of '+ portfolio.name)
    plt.grid(True)
    plt.legend(loc='best', fontsize = 12)
    ;
    return

In [ ]:
plot_var(SP500, 5)

In [ ]:
plot_var(US_treas, 5)

We can also find the VaR for many confidence levels:

In [ ]:
def value_at_risk(portfolio):
    
    percs = [0.01, 0.1, 1., 2.5, 5.0, 10.0]
    VaR = scs.scoreatpercentile(portfolio, percs)
    print("%16s %16s" % ('Confidence Level', 'Value-at-Risk'))
    print(33 * "-")
    for pair in zip(percs, -VaR):
        print("%11.2f %17.3f" % (100-pair[0], -pair[1]))
    return

In [ ]:
value_at_risk(SP500)

In [ ]:
value_at_risk(US_treas)

In [ ]:
percs = np.linspace(0,15)
SP500_var = scs.scoreatpercentile(SP500, percs)
US_treas_var = scs.scoreatpercentile(US_treas, percs)

In [ ]:
fig = plt.figure(figsize = (18,6))
plt.plot(percs, -SP500_var, lw=2, label='S&P 500', c='r')
plt.plot(percs, -US_treas_var, lw=2, label='US Treasury', c='b')
plt.legend(loc='best')
plt.xlabel('P (%)')
plt.ylabel('Value-at-risk')
plt.title('Value-at-risk')
plt.grid(True)
plt.autoscale(enable=True, axis='x', tight=True)

For each $p$, VaR of the S&P is always higher than VaR of US Treasuries.

## 2. Monte Carlo simulations

Given the limitations of the above methods, a good replacement for VaR analysis is to perform Monte Carlo (MC) simulations.
The idea behind MC simulation is that we replicate market outcomes on the computer, based on some model of the evolution of the market. By doing a sufficient number of simulations, we get a large sample of market outcomes enabling us to calculate accurately some quantities of interests (VaR in this case). The main limitation of this approach is that it is always based on some model and the quality of the results in inevitably limited by the quality of the model.

For ilustration purposes, let's generate 3 different returns distributions and see how VaR looks like for each one. The first two -standard normal and Student-t - are the most common one for replicating returns whereas the third one -uniform distributed- is an unusual case.

In [ ]:
# Standard Normal
norm = np.random.standard_normal(10000)

# Student t
t = np.random.standard_t(5,10000)

#Uniform
unif = np.random.uniform(-1,1,10000)

In [ ]:
fig = plt.figure(figsize = (22,6))
plt.subplot(1,3,1)
plt.hist(norm, color = 'blue', bins = 50)
plt.title('Standard normal')
plt.axvline(scs.scoreatpercentile(norm, 5), color='r', linestyle='dashed', linewidth=3)
plt.grid(True)

plt.subplot(1,3,2)
plt.hist(t, color = 'orange', bins = 50)
plt.title('Student-t')
plt.axvline(scs.scoreatpercentile(t, 5), color='r', linestyle='dashed', linewidth=3)
plt.grid(True)

plt.subplot(1,3,3)
plt.hist(unif, color = 'green', bins = 50)
plt.title('Uniform')
plt.axvline(scs.scoreatpercentile(unif, 5), color='r', linestyle='dashed', linewidth=3)
plt.grid(True)
;

Let's now generate daily returns of 3 assets, with the same Stundent-t distribution but with different degrees of freedom (df). As the degrees of freedom increase, fewer extreme observations are obtained.

If df $\rightarrow \infty$, Student-t $\rightarrow$ Normal.

In [ ]:
rets1 = np.random.standard_t(5,10000)*22/252
rets2 = np.random.standard_t(50,10000)*22/252
rets3 = np.random.standard_t(100,10000)*22/252
rets = [rets1, rets2, rets3]

In [ ]:
rets

Let's create a portfolio and calculate the VaR.

In [ ]:
weights = [1/3, 1/3, 1/3]
port_rets = np.dot(weights, rets)
port_rets = pd.Series(port_rets * initial_investment, name ='simulated portfolio')

In [ ]:
plot_var(port_rets, 5)

In [ ]:
value_at_risk(port_rets)

## Expected shortfall (ES)

One of the flaws of the VaR aproach is that it is simply a quantile. In practice, the actual loss, if it occurs, can be grater than VaR. In this sense, VaR may understimate the actual loss.
The most common alternative risk measure that tries to fill that gap is **expected shortfall** (ES), also known as tail VaR, conditional Value at Risk (CVaR), among others. ES answers the question: *what is expected loss when losses exceed VaR?*

####  Definition: 
*Expected loss conditional on VaR being violated. $ES = -E[Q|Q\leq-VaR(p)]$*


Let's define a function for ES and add it to our previous plots.

In [ ]:
def ES(portfolio, p):
    VaR = scs.scoreatpercentile(portfolio, p)
    shortfall = portfolio[portfolio<VaR]
        
    return shortfall.mean()

In [ ]:
mask1 = port_rets < scs.scoreatpercentile(port_rets, 5)
10000/len(port_rets[mask1])

In [ ]:
def plot_var_ES(portfolio, p):
    
    VaR = scs.scoreatpercentile(portfolio, p)
    ExpS = ES(portfolio, p)
    
    fig = plt.figure(figsize = (18,7))
    
    mask1 = portfolio < VaR
    mask2 = portfolio> VaR
    
    plt.hist(portfolio[mask2], bins = 60)
    plt.hist(portfolio[mask1], bins = 25, color='red')

    plt.axvline(VaR, color='r', linestyle='dashed', linewidth=2, 
                label = 'VaR at 5% ='+ str(round(VaR,2)))
    plt.axvline(ExpS, color='green', linestyle='dashdot', linewidth=2, 
                label = 'Expected shortfall = '+str(round(ES(portfolio,p),2)))
    
    plt.title('Distribution of '+ portfolio.name)
    plt.grid(True)
    plt.legend(loc='best', fontsize = 12)
    plt.xlabel('P&L')
    plt.ylabel('Frequency')
    plt.xlim((-3000, 3000))
    ;
    return

In [ ]:
plot_var_ES(SP500, 5)
plot_var_ES(US_treas, 5)
plot_var_ES(port_rets, 5)